# 4. Loading Data into DataFrames

In [ ]:
from dotenv import load_dotenv
import os
import settings
import pandas as pd
from sqlalchemy import create_engine
from tqdm.notebook import tqdm
import dask.dataframe as dd

load_dotenv(override=True)

DATABASE_URI = f"postgresql://{settings.DB_USERNAME}:{settings.DB_PASSWORD}@{settings.DB_HOST}/{settings.DB_NAME}"
DATABASE_URI


Pandas ile database'ten veri çekme

In [ ]:
dfl = []


def process_sql_using_pandas():
    engine = create_engine(DATABASE_URI)
    # tüm veriyi hafızaya atmamak için stream yapıyoruz.
    conn = engine.connect().execution_options(stream_results=True)

    for chunk_dataframe in tqdm(
        pd.read_sql_query(
            "SELECT id,title,description FROM public.article", conn, chunksize=1000
        )
    ):
        print(f"Got dataframe w/{len(chunk_dataframe)} rows")
        dfl.append(chunk_dataframe)

    return pd.concat(dfl)


df = process_sql_using_pandas()
df

Dask ile veri çekme

In [ ]:
data = dd.read_sql_table(
    "article",
    DATABASE_URI,
    index_col="id",
    npartitions=10,
    columns=["id", "title", "description"],
)
data


In [ ]:
df = data.compute()
df